In [1]:
# nltk.download('punkt') #uncomment if running on new machine

In [247]:
# Importing the libraries
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.externals import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gensim
from gensim.test.utils import datapath
from gensim.models import Word2Vec
import os
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize

# Helpful variables
EXT_DATA_FOLDER = "C:\\Users\\Admin\\Projects\\thesis\\data\\"
EXT_DATA_FOLDER2 = "B:\\Datasets\\"

ANALYSIS_SAMPLES = os.path.join(EXT_DATA_FOLDER, "Credibility_Analysis_Samples\\September_25\\")
dataset_columns = ['Identifier', 'Type', 'Category', 'URL', 'Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5',
 'Cat6', 'Cat7', 'Score', 'First date_time', 'Tweets', 'Likes', 'Retweets',
 'Potential exposure', 'HTML', 'TEXT']
criterias = ["Cat1", "Cat2", "Cat3", "Cat4", "Cat5", "Cat6", "Cat7"]


In [16]:
labelled_articles.keys()

Index(['Cat1', 'Cat2', 'Cat3', 'Cat4', 'Cat5', 'Cat6', 'Cat7', 'Category',
       'First date_time', 'HTML', 'Identifier', 'Likes', 'Potential Exposure',
       'Potential exposure', 'Retweets', 'Score', 'TEXT', 'Tweets', 'Type',
       'URL', 'Unnamed: 0', 'Low Credibility'],
      dtype='object')

In [4]:
art_text_sent = np.array([sent_tokenize(article.split("TITLE: ")[1].replace("TEXT: ","").strip(" ")) for article in labelled_articles["TEXT"]])
art_text_word = np.array([word_tokenize(article.split("TITLE: ")[1].replace("TEXT: ","").strip(" ")) for article in labelled_articles["TEXT"]])
art_text_sent_word = np.array([[word_tokenize(sent) for sent in article] for article in art_text_sent])

### Example Classifier Code

In [ ]:
for criteria in criterias:
    print("Training Classifier for " + criteria + ":")
    start_time = timeit.default_timer()

    X_train, X_test, y_train, y_test = train_test_split(list(labelled_articles["TEXT"]), list(labelled_articles[criteria]), test_size=int(20))

    svm_clf = Pipeline([('vect', CountVectorizer(max_features=10)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=1e-3, random_state=69,
                                               maxj_iter=5, tol=None)),
    ])
    svm_clf.fit(X_train, y_train)
    svm_predicted = list(svm_clf.predict(X_test))
    svm_cv_scores = cross_val_score(svm_clf, X_train, y_train, cv=10, scoring='f1_micro')

    print("SVM Average micro f1-score: %0.2f (+/- %0.2f)" % (svm_cv_scores.mean(), svm_cv_scores.std()))
    joblib.dump(svm_clf, 'models/svm_clf_tfidf_stop_' + criteria + '.joblib')

### Low Credibility Score Detector

1) Create a dataset that separates he articles as having low credibility and not low credibility where low credibility is defined to be articles with a credibility score of less than 3

2.a) Apply the ensemble of models to classify an article for each category and then use the resulting score to determine whether an article's credibility is low or not

2.b) Apply a new model that has been trained to solely just determine whether an article's credibility is low or not

3) ???

4) Profit

### Retrieving the manually labelled articles from the corpus

In [228]:
from collections import defaultdict

labelled_articles = pd.read_excel("dataset5.xlsx")
labelled_articles = labelled_articles.dropna(subset=['TEXT'])
labelled_articles = labelled_articles[pd.to_numeric(labelled_articles['Cat1'], errors='coerce').notnull()]
for criteria in criterias:
    labelled_articles = labelled_articles.dropna(subset=[criteria])
print(labelled_articles.shape)

#If score < 3 than article isn't credible
cred_thresh = 3
labelled_articles['Credible'] = np.where(labelled_articles['Score'] < cred_thresh, 0, 1)

(470, 21)


### Making Training and testing sets for 2 classification tasks

1) Make a training and testing set for classfying whether an article has low credibility or not

2) Make a training and testing set for classifying whether an article satisfies each criteria and then using these outputs do determine whether the article has low crediibility

In [214]:
article_text = [article.split("TITLE: ")[1].replace("TEXT: ","").strip(" ") for article in labelled_articles["TEXT"]]
crit1_labels = labelled_articles["Cat1"]
crit2_labels = labelled_articles["Cat2"]
crit3_labels = labelled_articles["Cat3"]
crit4_labels = labelled_articles["Cat4"]
crit5_labels = labelled_articles["Cat5"]
crit6_labels = labelled_articles["Cat6"]
crit7_labels = labelled_articles["Cat7"]
low_cred_labels = list(labelled_articles["Credible"])

In [185]:
print(len(labelled_articles[labelled_articles['Low Credibility'] == 1]))

112


### Single Model Approach
SVM + TF-IDF with stopwords removed

In [156]:
def svm_build_train(article_text, labels, seed=42):
    X_train, X_test, y_train, y_test = train_test_split(list(article_text), [int(label) for label in labels], random_state=seed, test_size=0.10)
    svm_clf = Pipeline([('vect', CountVectorizer(stop_words='english', analyzer='word', max_features=20000)),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                               alpha=1e-3, random_state=69,
                                               max_iter=5, tol=None)),
    ])
    svm_clf.fit(X_train, y_train)
    model = {"model": svm_clf,
             "X_train": X_train, 
             "X_test": X_test,
             "y_train": y_train,
             "y_test": y_test}

    return model

print(model.keys())

dict_keys(['model', 'X_train', 'X_test', 'y_train', 'y_test'])


In [249]:
models = defaultdict(str)
seed = random.randint(1,10000)
#Train a classifier for all 7 criteria
for criteria in criterias:
    models[criteria] = svm_build_train(article_text, labelled_articles[criteria], seed)

#Replicate the X and y test sets to get the actual credibility score
X2_train, X2_test, y2_train, credibility_scores = train_test_split(article_text, list(labelled_articles["Score"]), random_state=seed, test_size=0.10)
X2_train, X2_test, y2_train, low_credibility_labels = train_test_split(article_text, list(labelled_articles["Credible"]), random_state=seed, test_size=0.10)

#for each criteria, I get the use the model that I trained specifically for it and get it to predict on the articles stored in the X_test set
#The results are then stored in 'label_predictions' where it is a 7xN matrix where N is the number of articles in the test sets
label_predictions = []
for criteria in criterias:
    predictions = list(models[criteria]['model'].predict(models[criteria]['X_test']))
    label_predictions.append(predictions)

#The predictions are then tallied up and compared with the actual credibility score of the article
#And the score difference when it correctly and incorrectly identifies
predicted_scores = [0 for i in range(len(y_test))]
for label in label_predictions:
    for article in range(len(label)):
        predicted_scores[article] = predicted_scores[article] + label[article]

print("Predicted Scores vs Actual Scores:\n{}\n{}".format(predicted_scores, credibility_scores))

identified = []
correct_diff = []
incorrect_diff = []
for i in range(len(credibility_scores)):
    if((credibility_scores[i] < cred_thresh and predicted_scores[i] < cred_thresh) or (credibility_scores[i] >= cred_thresh and predicted_scores[i] >= cred_thresh)) :
        identified.append(1)
        correct_diff.append(abs(credibility_scores[i] - predicted_scores[i]))
    else:
        incorrect_diff.append(abs(credibility_scores[i] - predicted_scores[i]))

identified_avg = len(identified)/len(credibility_scores)
print("% Correct: {}".format(identified_avg))
correct_diff_avg = np.array(correct_diff).mean()
incorrect_diff_avg = np.array(incorrect_diff).mean()
print("Average difference when correct: {}\nAverage difference when incorrect: {}".format(correct_diff_avg, incorrect_diff_avg))

#Calculating the micro averaged f1 score

#Converting the predicted scores into a credibility label
low_credibility_preds = []
for i in range(len(credibility_scores)):
        if((credibility_scores[i] < cred_thresh and predicted_scores[i] < cred_thresh)):
            low_credibility_preds.append(0)
        else:
            low_credibility_preds.append(1)
        
print("Micro averaged f1-scores: {}".format(f1_score(low_credibility_labels, low_credibility_preds, labels=[0,1], average='micro')))
print("Micro averaged precision scores: {}".format(precision_score(low_credibility_labels, low_credibility_preds, labels=[0,1], average='micro')))
print("Micro averaged recall scores: {}".format(recall_score(low_credibility_labels, low_credibility_preds, labels=[0,1], average='micro')))

Predicted Scores vs Actual Scores:
[4, 4, 1, 4, 3, 1, 1, 3, 3, 4, 4, 4, 5, 3, 6, 4, 2, 1, 4, 4, 4, 2, 4, 4, 4, 4, 3, 4, 5, 4, 6, 4, 3, 4, 3, 5, 2, 3, 3, 3, 2, 4, 4, 4, 3, 4, 5]
[5, 5, 1, 4, 5, 1, 2, 5, 5, 4, 5, 4, 4, 1, 3, 4, 2, 2, 4, 4, 5, 5, 3, 3, 3, 1, 4, 5, 3, 4, 5, 4, 4, 3, 3, 4, 3, 2, 5, 5, 3, 4, 6, 4, 6, 4, 6]
% Correct: 0.8723404255319149
Average difference when correct: 0.9024390243902439
Average difference when incorrect: 1.8333333333333333
Micro averaged f1-scores: 0.9361702127659575
Micro averaged precision scores: 0.9361702127659575
Micro averaged recall scores: 0.9361702127659575


In [230]:
"""
svm_predicted = list(svm_clf.predict(X_test))
svm_cv_scores = cross_val_score(svm_clf, X_train, y_train, cv=10, scoring='f1_micro')

print("SVM Average micro f1-score: %0.2f (+/- %0.2f)" % (svm_cv_scores.mean(), svm_cv_scores.std()))
joblib.dump(svm_clf, 'models/svm_clf_tfidf_stop_' + criteria + '.joblib')
"""

'\nsvm_predicted = list(svm_clf.predict(X_test))\nsvm_cv_scores = cross_val_score(svm_clf, X_train, y_train, cv=10, scoring=\'f1_micro\')\n\nprint("SVM Average micro f1-score: %0.2f (+/- %0.2f)" % (svm_cv_scores.mean(), svm_cv_scores.std()))\njoblib.dump(svm_clf, \'models/svm_clf_tfidf_stop_\' + criteria + \'.joblib\')\n'

### Multiple runs of test to get idea of average accuracy and average difference between scores

In [255]:
import random

total_identified = []
total_correct_diff = []
total_incorrect_diff = []
total_low_cred_preds = []
total_low_cred_labels = []
num_runs = 500
for i in range(num_runs):
    models = defaultdict(str)
    seed = random.randint(1,100000)
    #Train a classifier for all 7 criteria
    for criteria in criterias:
        models[criteria] = svm_build_train(article_text, labelled_articles[criteria], seed)

    #Replicate the X and y test sets to get the actual credibility score
    X2_train, X2_test, y2_train, credibility_scores = train_test_split(article_text, list(labelled_articles["Score"]), random_state=seed, test_size=0.10)

    #for each criteria, I get the use the model that I trained specifically for it and get it to predict on the articles stored in the X_test set
    #The results are then stored in 'label_predictions' where it is a 7xN matrix where N is the number of articles in the test sets
    label_predictions = []
    for criteria in criterias:
        predictions = list(models[criteria]['model'].predict(models[criteria]['X_test']))
        label_predictions.append(predictions)

    #The predictions are then tallied up and compared with the actual credibility score of the article
    #And the score difference when it correctly and incorrectly identifies
    predicted_scores = [0 for i in range(len(y_test))]
    for label in label_predictions:
        for article in range(len(label)):
            predicted_scores[article] = predicted_scores[article] + label[article]
    identified = []
    correct_diff = []
    incorrect_diff = []
    for i in range(len(credibility_scores)):
        if((credibility_scores[i] < 3 and predicted_scores[i] < 3) or (credibility_scores[i] >= 3 and predicted_scores[i] >= 3)) :
            identified.append(1)
            correct_diff.append(abs(credibility_scores[i] - predicted_scores[i]))
        else:
            incorrect_diff.append(abs(credibility_scores[i] - predicted_scores[i]))

    identified_avg = len(identified)/len(credibility_scores)
    correct_diff_avg = np.array(correct_diff).mean()
    incorrect_diff_avg = np.array(incorrect_diff).mean()
    
    total_identified.append(identified_avg)
    total_correct_diff.append(correct_diff_avg)
    total_incorrect_diff.append(incorrect_diff_avg)
    
    #Calculating the micro averaged f1 score

    #Converting the predicted scores into a credibility label
    low_credibility_preds = []
    for i in range(len(credibility_scores)):
            if((credibility_scores[i] < cred_thresh and predicted_scores[i] < cred_thresh)):
                low_credibility_preds.append(0)
            else:
                low_credibility_preds.append(1)
    total_low_cred_preds.extend(low_credibility_preds)
    total_low_cred_labels.extend(low_credibility_labels)
    
print("""
For {} runs:\n
Average correctly identified %: {}\n
Average score difference when correctly classified: {}\n
Average score difference when incorrectly classified: {}
""".format(num_runs, np.array(total_identified).mean(), np.array(total_correct_diff).mean(), np.array(total_incorrect_diff).mean()))



print("Micro averaged f1-scores: {}".format(f1_score(total_low_cred_labels, total_low_cred_preds, labels=[0,1], average='micro')))
print("Micro averaged precision scores: {}".format(precision_score(total_low_cred_labels, total_low_cred_preds, labels=[0,1], average='micro')))
print("Micro averaged recall scores: {}".format(recall_score(total_low_cred_labels, total_low_cred_preds, labels=[0,1], average='micro')))


For 500 runs:

Average correctly identified %: 0.8780851063829787

Average score difference when correctly classified: 0.7126854265117839

Average score difference when incorrectly classified: 1.9099660894660895

Micro averaged f1-scores: 0.7208936170212767
Micro averaged precision scores: 0.7208936170212766
Micro averaged recall scores: 0.7208936170212766
